In [6]:
import json
import numpy as np
from multiprocessing import Pool
from functools import partial
from itertools import product
from scipy.special import logsumexp
import itertools

In [ ]:
n = 3
x = itertools.product([0, 1], repeat=n) 

def get_partition_function(energy, multiplicity, T=298.15, return_pi=True, N_N=0, N_potential=0.):
    k_b = 8.617333262145e-05  # Boltzmann constant in eV/K
    energy = np.array(energy, dtype=float)
    multiplicity = np.array(multiplicity, dtype=float)

    # Compute log probabilities
    log_p_i = np.log(multiplicity) + (-energy + (N_N * N_potential)) / (k_b * T)
    log_Z = logsumexp(log_p_i)  # log of partition function

    if not np.isfinite(log_Z):
        print("Warning: log_Z is invalid!", log_Z)

    # Normalize in log-space and then exponentiate
    log_pi = log_p_i - log_Z
    pi = np.exp(log_pi)

    if return_pi:
        return np.exp(log_Z), pi
    return np.exp(log_Z)

# Load the energies and concentrations
with open("sample_4x3_4um.json", "r") as f:
    data = json.load(f)

Ryd_classical_E = np.array(data["Rydberg_classical_energy"])
concentration_all = np.array(data["concentration"])
num_sites = 28

# Define Δμ and T ranges
eV_to_rad_s = 1.519267447321156e15
Delta_mu_range = np.linspace(-1.25e8, 1.25e8, 100, endpoint=True)
Delta_mu_range = Delta_mu_range / eV_to_rad_s

delta_g_all = ['125000000','62500000','31250000','15625000','0','-31250000']
# delta_g_all_int = [-int(x) for x in delta_g_all]

C6 = 5.42e-24
reg_coef = [0.00030122447260882713, 0.00020350836925458785]
V_4um = (C6/(4.01e-6**6))
V_qubo = reg_coef[1]*eV_to_rad_s
V_ratio = V_qubo/V_4um

delta_g_all_int = np.array([-int(x) for x in delta_g_all])
Delta_mu_range = delta_g_all_int / eV_to_rad_s

T_all = 5e-5



for Delta_mu_str, Delta_mu in zip(delta_g_all, Delta_mu_range):
    prob_site_mu = []

    energy_new = Ryd_classical_E + concentration * Delta_mu
    Z, pi = get_partition_function(
        energy_new, [1] * len(Ryd_classical_E), return_pi=True, T=T
    )

    for i, x in enumerate(itertools.product([0, 1], repeat=len(coordinates))):
        x_arr = np.array(x)
        if np.sum(x_arr) > 0:
            prob_site = (pi[i] * x_arr) / np.sum(x_arr)
        else:
            prob_site = np.zeros_like(x_arr)

        prob_site_mu.append(prob_site.tolist())  # convert to list for JSON compatibility

    results[Delta_mu_str] = prob_site_mu




NameError: name 'itertools' is not defined

In [10]:
n = 2
x = itertools.product([0, 1], repeat=n) 
for xx in x:
    print(xx)

(0, 0)
(0, 1)
(1, 0)
(1, 1)
